# Downloading issues and pull requests.

This document provides a basis for researching Github data locally.  It uses the a repository's metadata to access the Github Issues and Pull Requests.

In [46]:
    import pandas; import requests; from functools import partial
    from requests_cache import install_cache
    install_cache('github-data')

In [47]:
    info = lambda x, **params: pandas.Series(requests.get("https://api.github.com/repos/{}".format(x), params=params).json())

In [48]:
    events = lambda action, x, **params: pandas.DataFrame(requests.get(
        info(x, **params).loc[f'{action}_url'].format(**{'/number': ''})).json())

In [1]:
    def pulls(
        project: str, 
        iter: int=2, 
        state: ('open', 'closed', 'all')='closed'
    ) -> iter:
        """Download the pull requests over {iter} pages for specific state of
        pull request"""
        for i in range(1, iter):
            result = events('pulls', project, state=state, page=i)
            yield result
            if len(result) < 30: break


In [2]:
    def test_jupyterlab(): 
        assert len(pandas.concat(list(pulls('jupyterlab/jupyterlab'))))

In [4]:
    if __name__ == '__main__':
        !ipython -m pytest -- 2018-08-25-Moving-github-data-into-dataframes.ipynb

============================= test session starts =============================
platform win32 -- Python 3.6.5, pytest-3.5.1, py-1.5.3, pluggy-0.6.0
Matplotlib: 2.2.2
Freetype: 2.8.1
rootdir: C:\Users\deathbeds\deathbeds.github.io\deathbeds, inifile: tox.ini
plugins: xdist-1.22.5, testmon-0.9.12, remotedata-0.2.1, parallel-0.0.2, openfiles-0.3.0, mpl-0.9, localserver-0.4.1, forked-0.2, doctestplus-0.1.3, arraydiff-0.2, hypothesis-3.66.16, importnb-0.5.0
collected 1 item

2018-08-25-Moving-github-data-into-dataframes.ipynb .                    [100%]

========================== 1 passed in 9.00 seconds ===========================
